# scRFE anndatasubset

In [ ]:
# testing to see if works from pypi - Angela error from 06/16/20
# And fixing scRFE to have groups A and B have the same # of cells in both
# by subsetting B cells to equal # of A or having a max cutoff for both 

In [84]:
# MENTION ONE VS ALL CLASSIFICATION in description

In [48]:
# Imports 
import numpy as np
import pandas as pd
import scanpy as sc
import random
from anndata import read_h5ad
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
import seaborn as sns
import matplotlib.pyplot as plt
import scanpy.external as sce
import logging as logg

In [138]:
adata = read_h5ad('/Users/madelinepark/Downloads/Kidney_facs.h5ad')

# tflist = pd.read_csv('/Users/madelinepark/Downloads/GO_term_summary_20171110_222852.csv')
# adata_tfs = adata[:,adata.var_names.isin(tflist['Symbol'])].copy()

In [203]:
adataHeart = read_h5ad('/Users/madelinepark/Downloads/Heart_droplet.h5ad')


In [218]:
adataLiver = read_h5ad('/Users/madelinepark/Downloads/Liver_droplet.h5ad')

In [2]:
def filterNormalize (dataMatrix, classOfInterest):
    np.random.seed(644685)
    sc.logging.print_versions()
    sc.settings.verbosity = 3      
    sc.logging.print_versions()
    tiss = dataMatrix
    tiss.obs['n_counts'] = tiss.X.sum(axis=1).A1
    sc.pp.filter_cells(tiss, min_genes=250)
    sc.pp.filter_genes(tiss, min_cells=3)
    tiss = tiss[tiss.obs['n_counts'] > 1500, :]
    sc.pp.normalize_per_cell(tiss, counts_per_cell_after=1e5)
    sc.pp.log1p(tiss)
    tiss.raw = tiss
    tiss = tiss[tiss.obs[classOfInterest]!='nan']
    return tiss

In [31]:
# goal: get labels on a per class basis that will go into randomForest function for y
def getLabels (dataMatrix, classOfInterest): 
    """
    Gets labels on a per class basis that will inputted to the randomForest function
    
    Parameters
    ----------
    dataMatrix : anndata object
        The data file of interest
    classOfInterest : str
        The class you will split the data by in the set of dataMatrix.obs
    
    Returns
    -------
    labelsDict : dict
        Dictionary with labels for each class 
    """
    dataMatrix = filterNormalize (dataMatrix, classOfInterest)
    labelsDict = {}
    for label in np.unique(dataMatrix.obs[classOfInterest]):
        lists = []        
        for obs in dataMatrix.obs[classOfInterest]:
            if obs == label: 
                lists.append('A')
            else:
                lists.append('B')
        labelsDict[label] = lists #this is usually in line w if and else
    return labelsDict

In [6]:
print((labelsDict['3m']).count('A'))

509


In [155]:
prints1 = [
]
prints2 = []
for obs in adataHeart1.obs['age']:
    if obs == '24m':
        prints1.append(1)
    else:
        prints2.append(2)
print(len(prints1))
print(len(prints2))

744
7172


In [165]:
adataHeart1 = adataHeart
adataHeart1.obs['classification_group'] = 'B'
adataHeart1.obs.loc[adataHeart1.obs[adataHeart1.obs['age']=='24m'].
                    index,'classification_group'] = 'A'

In [166]:
dataMatrix2 = downsampleToSmallestCategory(dataMatrix = adataHeart1, 
                                             classOfInterest = 'classification_group', 
                                             keep_small_categories = True)

IndexError: index 5 is out of bounds for axis 0 with size 5

In [167]:
lists11 = []
lists22 = []
for obs in dataMatrix2.obs['classification_group']:
    if obs == 'A':
        lists1.append('1')
    else:
        lists2.append('2')
print(len(lists11))
print(len(lists22))

0
0


In [ ]:
adata1 = adata
adata1.obs['classification_group'] = 'B'
adata1.obs.loc[adata1.obs[adata1.obs['age']=='24m'].index,'classification_group'] = 'A'
adata1.obs

In [163]:
dataMatrix1 = downsampleToSmallestCategory(dataMatrix = adata1, 
                                             classOfInterest = 'classification_group', 
                                             keep_small_categories = True)

In [164]:
lists1 = []
lists2 = []
for obs in dataMatrix1.obs['classification_group']:
    if obs == 'A':
        lists1.append('1')
    else:
        lists2.append('2')
print(len(lists1))
print(len(lists2))

561
561


In [99]:
def downsampleToSmallestCategory(dataMatrix, classOfInterest,
                                 random_state = None,
                                 min_cells = 15, 
                                 keep_small_categories = False) -> sc.AnnData: 
    """
    returns an annData object in which all categories in 'classOfInterest' have
    the same size
    classOfInterest
        classOfInterest with the categories to downsample
    min_cells
        Minimum number of cells to downsample.
        Categories having less than `min_cells` are discarded unless
        keep_small_categories is True
    keep_small_categories
        Be default categories with less than min_cells are discarded.
        Set to true to keep them
    """
    counts = dataMatrix.obs[classOfInterest].value_counts(sort=False)
    if len(counts[counts < min_cells]) > 0 and keep_small_categories is False:
        logg.warning(
            "The following categories have less than {} cells and will be "
            "ignored: {}".format(min_cells, dict(counts[counts < min_cells]))
        )
    min_size = min(counts[counts >= min_cells])
    sample_selection = None
    for sample, num_cells in counts.items():
        if num_cells <= min_cells:
            if keep_small_categories:
                sel = dataMatrix.obs.index.isin(
                    dataMatrix.obs[dataMatrix.obs[classOfInterest] == sample].index)
            else:
                continue
        else:
            sel = dataMatrix.obs.index.isin(
                dataMatrix.obs[dataMatrix.obs[classOfInterest] == sample]
                .sample(min_size, random_state=random_state)
                .index
            )
        if sample_selection is None:
            sample_selection = sel
        else:
            sample_selection |= sel
    logg.info(
        "The cells in category {!r} had been down-sampled to have each {} cells. "
        "The original counts where {}".format(classOfInterest, min_size, dict(counts))
    )
    return dataMatrix[sample_selection].copy()

In [129]:
# dataMatrix1.obs['classification_group'].value_counts
(dataMatrix1.obs['classification_group']).count('A')

KeyError: 'Level A must be same as name (index)'

In [47]:
def makeOneForest (dataMatrix, classOfInterest, labelOfInterest, nEstimators = 5000, 
                   randomState = 0,  nJobs = -1, oobScore = True, Step = 0.2, Cv = 5): 
    """
    Builds and runs a random forest for one label in a class of interest
    
    Parameters
    ----------
    dataMatrix : anndata object
        The data file of interest
    classOfInterest : str
        The class you will split the data by in the set of dataMatrix.obs
    labelOfInterest : str
        The specific label within the class that the random forezt will run a 
        "one vs all" classification on
    nEstimators : int
        The number of trees in the forest
    randomState : int
        Controls random number being used
    nJobs : int
        The number of jobs to run in parallel
    oobScore : bool
        Whether to use out-of-bag samples to estimate the generalization accuracy
    Step : float
        Corresponds to percentage of features to remove at each iteration
    Cv : int
        Determines the cross-validation splitting strategy
        
    Returns
    -------
    feature_selected : list
        list of top features from random forest
    selector.estimator_.feature_importances_ : list
        list of top ginis corresponding to to features
    
    """
    dataMatrix = filterNormalize (dataMatrix, classOfInterest)

#     print('makeOneForest' + labelOfInterest)
    labelsDict = getLabels(dataMatrix, classOfInterest) 

    feat_labels = dataMatrix.var_names #this is equivalent of the genes
    X = dataMatrix.X
    y = labelsDict[labelOfInterest]
#     print('Y')
    clf = RandomForestClassifier(n_estimators = nEstimators, random_state = randomState, 
                                 n_jobs = nJobs, oob_score = oobScore)
    selector = RFECV(clf, step = Step, cv = Cv)
    
#     print('training...')
    clf.fit(X, y)
    selector.fit(X, y)
    feature_selected = feat_labels[selector.support_] 

    return feature_selected, selector.estimator_.feature_importances_ 

In [5]:
def resultWrite (classOfInterest, results_df, labelOfInterest,
                feature_selected, feature_importance):
#     print ('result writing')
#     print(results_df)
    
    column_headings = [] 
    column_headings.append(labelOfInterest)
    column_headings.append(labelOfInterest + '_gini')
    resaux = pd.DataFrame(columns = column_headings)
    resaux[labelOfInterest] = feature_selected
    resaux[labelOfInterest + '_gini'] = feature_importance
    resaux = resaux.sort_values(by = [labelOfInterest + '_gini'], ascending = False)
    resaux.reset_index(drop = True, inplace = True)

    results_df = pd.concat([results_df, resaux], axis=1)
    return results_df 

In [6]:
def scRFE(dataMatrix, classOfInterest, nEstimators = 5000, randomState = 0,  
                  nJobs = -1, oobScore = True, Step = 0.2, Cv = 5):
    """
    Builds and runs a random forest with one vs all classification for each label 
    for one class of interest
    
    Parameters
    ----------
    dataMatrix : anndata object
        The data file of interest
    classOfInterest : str
        The class you will split the data by in the set of dataMatrix.obs
    labelOfInterest : str
        The specific label within the class that the random forezt will run a 
        "one vs all" classification on
    nEstimators : int
        The number of trees in the forest
    randomState : int
        Controls random number being used
    nJobs : int
        The number of jobs to run in parallel
    oobScore : bool
        Whether to use out-of-bag samples to estimate the generalization accuracy
    Step : float
        Corresponds to percentage of features to remove at each iteration
    Cv : int
        Determines the cross-validation splitting strategy
        
    Returns
    -------
    results_df : pd.DataFrame
        Dataframe with results for each label in the class, formatted as 
        "label" for one column, then "label + gini" for the corresponding column
    
    """
    
    dataMatrix = filterNormalize (dataMatrix, classOfInterest)
    results_df = pd.DataFrame()
    for labelOfInterest in np.unique(dataMatrix.obs[classOfInterest]): #for timeliness    
#         print( 'scRFE' + labelOfInterest)
        
        feature_selected, feature_importance = makeOneForest(dataMatrix, 
                                                             classOfInterest, 
                          labelOfInterest = labelOfInterest)
    
        results_df = resultWrite (classOfInterest, results_df, 
                            labelOfInterest = labelOfInterest, 
                    feature_selected = feature_selected,  
                    feature_importance = feature_importance)
#         print(results_df.shape)
    return results_df

In [40]:
def OlDmakeOneForest (dataMatrix, classOfInterest, labelOfInterest, nEstimators = 5000, 
                   randomState = 0,  nJobs = -1, oobScore = True, Step = 0.2, Cv = 5,
                   maxObs = 10000): #GONNA HAVE TO ADD THIS to documentation
    
    dataMatrix = filterNormalize (dataMatrix, classOfInterest)

    labelsDict = getLabels(dataMatrix, classOfInterest) 
    
    dataMatrixA = dataMatrix[dataMatrix.obs[classOfInterest]==labelOfInterest].copy()
    dataMatrixB = dataMatrix[~dataMatrix.obs[classOfInterest]==labelOfInterest].copy()

    if (labelsDict[labelOfInterest]).count('A') >= maxObs:
        sc.pp.subsample(data = dataMatrixA, fraction = None, n_obs = maxObs, 
                            random_state = 0, copy = False)
        sc.pp.subsample(data = dataMatrixB, fraction = None, n_obs = maxObs, 
                            random_state = 0, copy = False)
        
    elif (labelsDict[labelOfInterest]).count('A') <= maxObs :
        countA = (labelsDict[labelOfInterest]).count('A')
        sc.pp.subsample(data = dataMatrixA, fraction = None, n_obs = countA, 
                            random_state = 0, copy = False)
        sc.pp.subsample(data = dataMatrixB, fraction = None, n_obs = countA, 
                            random_state = 0, copy = False)
    
    feat_labels = dataMatrix.var_names 
    X = dataMatrix.X
    y = labelsDict[labelOfInterest]
    clf = RandomForestClassifier(n_estimators = nEstimators, random_state = randomState, 
                                 n_jobs = nJobs, oob_score = oobScore)
    selector = RFECV(clf, step = Step, cv = Cv)
    
    clf.fit(X, y)
    selector.fit(X, y)
    feature_selected = feat_labels[selector.support_] 

    return feature_selected, selector.estimator_.feature_importances_ 

In [42]:
# dataMatrixA = dataMatrix[dataMatrix.obs[classOfInterest]==labelOfInterest].copy()
dataMatrixA = adata[adata.obs['age']=='24m'].copy()

# dataMatrixB = dataMatrix[~dataMatrix.obs[classOfInterest]==labelOfInterest].copy()
dataMatrixB = adata[~adata.obs['age']=='24m'].copy()



TypeError: bad operand type for unary ~: 'str'

In [213]:
adataHeart.obs['age'] = adataHeart.obs['age'].astype(str)

In [220]:
adataHeart.obs['age'] = adataHeart.obs['age'].astype(str)
def labelSplit (dataMatrix, classOfInterest, labelOfInterest):
    dataMatrix.obs['classification_group'] = 'B'
    dataMatrix.obs.loc[dataMatrix.obs[dataMatrix.obs[classOfInterest]==labelOfInterest]
                   .index,'classification_group'] = 'A'
    return dataMatrix

In [ ]:
adataHeartSplit = labelSplit (dataMatrix = adataHeart, 
                              classOfInterest = 'age', labelOfInterest = '24m')
adataHeartSplit.obs

In [216]:
import logging as logg
def downsample_to_smallest_category(
        adata,
        column="sample_short",
        random_state=None,
        min_cells=15,
        keep_small_categories=False
) -> sc.AnnData:
    """
    returns an annData object in which all categories in 'column' have
    the same size
    column
        column with the categories to downsample
    min_cells
        Minimum number of cells to downsample.
        Categories having less than `min_cells` are discarded unless
        keep_small_categories is True
    keep_small_categories
        Be default categories with less than min_cells are discarded.
        Set to true to keep them
    """
    counts = adata.obs[column].value_counts(sort=False)
    if len(counts[counts < min_cells]) > 0 and keep_small_categories is False:
        logg.warning(
            "The following categories have less than {} cells and will be "
            "ignored: {}".format(min_cells, dict(counts[counts < min_cells]))
        )
    min_size = min(counts[counts >= min_cells])
    sample_selection = None
    for sample, num_cells in counts.items():
        if num_cells <= min_cells:
            if keep_small_categories:
                sel = adata.obs.index.isin(
                    adata.obs[adata.obs[column] == sample].index)
            else:
                continue
        else:
            sel = adata.obs.index.isin(
                adata.obs[adata.obs[column] == sample]
                .sample(min_size, random_state=random_state)
                .index
            )
        if sample_selection is None:
            sample_selection = sel
        else:
            sample_selection |= sel
    logg.info(
        "The cells in category {!r} had been down-sampled to have each {} cells. "
        "The original counts where {}".format(column, min_size, dict(counts))
    )
    return adata[sample_selection].copy()

In [ ]:
downsample_to_smallest_category(
        adata = adataHeartSplit,
        column='classification_group',
        random_state=None,
        min_cells=15,
        keep_small_categories=False)

In [219]:
adataLiver.obs['age'] = adataLiver.obs['age'].astype(str)

In [229]:
adataLiverSplit = labelSplit (dataMatrix = adataLiver, 
                              classOfInterest = 'age', labelOfInterest = '18m')

In [230]:
rex = downsample_to_smallest_category(
        adata = adataLiverSplit,
        column='classification_group',
        random_state=None,
        min_cells=15,
        keep_small_categories=False)

In [231]:
rex.obs

age batch  \
index                                    
AAAGCAATCGGAAATA-1-9-0-0     18m     0   
AAAGTAGAGGCCCTTG-1-9-0-0     18m     0   
AACCGCGAGAAACCGC-1-9-0-0     18m     0   
AACTCCCAGTTGTCGT-1-9-0-0     18m     0   
AACTCCCGTAACGTTC-1-9-0-0     18m     0   
AACTCTTCAAAGTCAA-1-9-0-0     18m     0   
AAGGAGCGTTAAAGAC-1-9-0-0     18m     0   
AAGGCAGTCTTTACAC-1-9-0-0     18m     0   
AAGGTTCAGTGGGCTA-1-9-0-0     18m     0   
ACACTGAAGTCGATAA-1-9-0-0     18m     0   
ACATCAGCAAGCGATG-1-9-0-0     18m     0   
ACATGGTAGTCCCACG-1-9-0-0     18m     0   
ACCAGTACATTGGGCC-1-9-0-0     18m     0   
ACCCACTAGCCCAATT-1-9-0-0     18m     0   
ACCTTTAAGTTATCGC-1-9-0-0     18m     0   
ACGAGCCCAGCCAATT-1-9-0-0     18m     0   
ACGATACCACATTAGC-1-9-0-0     18m     0   
ACGATACGTCTAAACC-1-9-0-0     18m     0   
ACGATGTAGTTTGCGT-1-9-0-0     18m     0   
ACGCAGCAGGACGAAA-1-9-0-0     18m     0   
ACGCCAGGTTACAGAA-1-9-0-0     18m     0   
ACGGGTCCATCGGACC-1-9-0-0     18m     0   
ACGTCAATCGCTTGTC-1-9-0-0     18m     0   
ACGTCAATCTGCCCTA-1-9-0-0     18m     0   
ACTATCTCATCAGTAC-1-9-0-0     18m     0   
ACTATCTGTGCCTTGG-1-9-0-0     18m     0   
ACTATCTGTGGTGTAG-1-9-0-0     18m     0   
ACTATCTTCATAGCAC-1-9-0-0     18m     0   
ACTGAACGTTCCGGCA-1-9-0-0     18m     0   
ACTGATGGTCAGGACA-1-9-0-0     18m     0   
...                          ...   ...   
10X_P7_0_CGCTTCAAGAGACTTA-1   3m     1   
10X_P7_0_CGTAGCGTCCGCAGTG-1   3m     1   
10X_P7_0_CTACACCCAGAGCCAA-1   3m     1   
10X_P7_0_CTACATTTCTGATTCT-1   3m     1   
10X_P7_0_CTACCCACAGCCAATT-1   3m     1   
10X_P7_0_CTCAGAACATATGCTG-1   3m     1   
10X_P7_0_GACTGCGCAATGAATG-1   3m     1   
10X_P7_0_GATCGATCAGCATGAG-1   3m     1   
10X_P7_0_GGGAGATCACTTCTGC-1   3m     1   
10X_P7_0_GTAACTGTCCTTGACC-1   3m     1   
10X_P7_0_GTGCTTCCAGACGCTC-1   3m     1   
10X_P7_0_GTTACAGCACCCAGTG-1   3m     1   
10X_P7_0_TGCCAAAAGTTTCCTT-1   3m     1   
10X_P7_0_TGGGCGTTCGTAGATC-1   3m     1   
10X_P7_0_TGGTTCCAGTGTCCCG-1   3m     1   
10X_P7_0_TTAGGACGTTCGTTGA-1   3m     1   
10X_P7_0_TTCGGTCCAAAGGCGT-1   3m     1   
10X_P7_1_ACACCAAGTCCAACTA-1   3m     1   
10X_P7_1_ACCTTTACAGCTCGAC-1   3m     1   
10X_P7_1_CAACCTCGTGGTGTAG-1   3m     1   
10X_P7_1_CACTCCATCACCTTAT-1   3m     1   
10X_P7_1_CAGTCCTTCCAAAGTC-1   3m     1   
10X_P7_1_CGATCGGAGAAACGCC-1   3m     1   
10X_P7_1_CGATGTATCGCAAGCC-1   3m     1   
10X_P7_1_GCAGCCAGTTGCGCAC-1   3m     1   
10X_P7_1_GCTGGGTGTCAGAAGC-1   3m     1   
10X_P7_1_GGCCGATGTCGCGTGT-1   3m     1   
10X_P7_1_GTATTCTCATGCCTAA-1   3m     1   
10X_P7_1_GTGCATAAGTCCGTAT-1   3m     1   
10X_P7_1_TAAGTGCTCATGTAGC-1   3m     1   

                                                                         cell  \
index                                                                           
AAAGCAATCGGAAATA-1-9-0-0     MACA_18m_F_LIVER_HEPATOCYTES_51_AAAGCAATCGGAAATA   
AAAGTAGAGGCCCTTG-1-9-0-0     MACA_18m_F_LIVER_HEPATOCYTES_51_AAAGTAGAGGCCCTTG   
AACCGCGAGAAACCGC-1-9-0-0     MACA_18m_F_LIVER_HEPATOCYTES_51_AACCGCGAGAAACCGC   
AACTCCCAGTTGTCGT-1-9-0-0     MACA_18m_F_LIVER_HEPATOCYTES_51_AACTCCCAGTTGTCGT   
AACTCCCGTAACGTTC-1-9-0-0     MACA_18m_F_LIVER_HEPATOCYTES_51_AACTCCCGTAACGTTC   
AACTCTTCAAAGTCAA-1-9-0-0     MACA_18m_F_LIVER_HEPATOCYTES_51_AACTCTTCAAAGTCAA   
AAGGAGCGTTAAAGAC-1-9-0-0     MACA_18m_F_LIVER_HEPATOCYTES_51_AAGGAGCGTTAAAGAC   
AAGGCAGTCTTTACAC-1-9-0-0     MACA_18m_F_LIVER_HEPATOCYTES_51_AAGGCAGTCTTTACAC   
AAGGTTCAGTGGGCTA-1-9-0-0     MACA_18m_F_LIVER_HEPATOCYTES_51_AAGGTTCAGTGGGCTA   
ACACTGAAGTCGATAA-1-9-0-0     MACA_18m_F_LIVER_HEPATOCYTES_51_ACACTGAAGTCGATAA   
ACATCAGCAAGCGATG-1-9-0-0     MACA_18m_F_LIVER_HEPATOCYTES_51_ACATCAGCAAGCGATG   
ACATGGTAGTCCCACG-1-9-0-0     MACA_18m_F_LIVER_HEPATOCYTES_51_ACATGGTAGTCCCACG   
ACCAGTACATTGGGCC-1-9-0-0     MACA_18m_F_LIVER_HEPATOCYTES_51_ACCAGTACATTGGGCC   
ACCCACTAGCCCAATT-1-9-0-0     MACA_18m_F_LIVER_HEPATOCYTES_51_ACCCACTAGCCCAATT   
ACCTTTAAGTTATCGC-1-9-0-0     MACA_18m_F_LIVER_HEPATOCYTES_51_ACCTTTAAGTTATCGC   
ACGAG

In [232]:
type(rex)

anndata.base.AnnData

In [233]:
lists1 = []
lists2 = []
for obs in rex.obs['classification_group']:
    if obs == 'A':
        lists1.append('1')
    else:
        lists2.append('2')
print(len(lists1))
print(len(lists2))

547
547


In [198]:
downsampleToSmallestCategory(dataMatrix = adataHeartSplit, 
                             classOfInterest = 'classification_group')

IndexError: index 5 is out of bounds for axis 0 with size 5

In [194]:
len(set(adataHeart.obs['age']))

6